In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Наложение и удаление шума

In [ ]:
image = cv2.imread('sar_1.jpg')
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
plt.imshow(image_gray, cmap="gray")

In [ ]:
# Gaussian noise
mean = 0
stddev = 100
noise_gauss = np.zeros(image_gray.shape, np.uint8)
cv2.randn(noise_gauss, mean, stddev)

In [ ]:
plt.imshow(noise_gauss, cmap="gray")

In [ ]:
# Salt and pepper
noise = np.random.randint(0, 101, size=(image_gray.shape[0], image_gray.shape[1]), dtype=int)
zeros_pixel = np.where(noise == 0)
ones_pixel = np.where(noise == 100)

In [ ]:
bg_image = np.ones(image_gray.shape, np.uint8) * 128

In [ ]:
bg_image[zeros_pixel] = 0
bg_image[ones_pixel] = 255

In [ ]:
plt.imshow(bg_image, cmap="gray")

In [ ]:
image_noise_gauss = cv2.add(image_gray, noise_gauss)

In [ ]:
plt.imshow(image_noise_gauss, cmap="gray")

In [ ]:
from skimage.metrics import structural_similarity, mean_squared_error

In [ ]:
mse_gauss = mean_squared_error(image_gray, image_noise_gauss)
(ssim, diff) = structural_similarity(image_gray, image_noise_gauss, full=True)
print(mse_gauss, ssim)

In [ ]:
image_gauss_median = cv2.medianBlur(image_noise_gauss, 3)

In [ ]:
mse_gauss_median = mean_squared_error(image_gray, image_gauss_median)
(ssim_gauss_median, diff) = structural_similarity(image_gray, image_gauss_median, full=True)

In [ ]:
print(mse_gauss_median, ssim_gauss_median)

In [ ]:
plt.imshow(image_gauss_median, cmap="gray")

In [ ]:
import copy

image_sp = copy.deepcopy(image_gray)

image_sp[zeros_pixel] = 0
image_sp[ones_pixel] = 255

In [ ]:
plt.imshow(image_sp, cmap="gray")

In [ ]:
mse_sp = mean_squared_error(image_gray, image_sp)
(ssim_sp, diff) = structural_similarity(image_gray, image_sp, full=True)
print(mse_sp, ssim_sp)

In [ ]:
image_sp_median = cv2.medianBlur(image_sp, 3)

In [ ]:
mse_sp_median = mean_squared_error(image_gray, image_sp_median)
(ssim_sp_median, diff) = structural_similarity(image_gray, image_sp_median, full=True)
print(mse_sp_median, ssim_sp_median)

# Другие типы фильтров

In [ ]:
image_gauss_gauss = cv2.GaussianBlur(image_noise_gauss, (5, 5), 0)

In [ ]:
image_gauss_bilat = cv2.bilateralFilter(image_noise_gauss, 9, 75, 75)

In [ ]:
image_gauss_nlm = cv2.fastNlMeansDenoising(image_noise_gauss, h=20)

In [ ]:
import math


def geom(a):
    prod = 1
    for i in range(a.shape[0]):
        prod1 = 1
        for j in range(a.shape[1]):
            prod1 *= a[i, j]
        prod1 = math.pow(prod1, 1.0 / 9.0)
        prod *= prod1
    return prod


def proc(img, filter):
    img_res = copy.deepcopy(img)
    for i in range(0, img.shape[0] - 2):
        for j in range(0, img.shape[1] - 2):
            img_res[i:i + 3, j:j + 3] = filter(img[i:i + 3, j:j + 3])
    return img_res


res = proc(image_noise_gauss, geom)


In [ ]:
plt.imshow(res, cmap="gray")

In [ ]:
mse_geom = mean_squared_error(image_gray, res)
(ssim_geom, diff) = structural_similarity(image_gray, res, full=True)
print(mse_geom, ssim_geom)


# 2D свертка

In [ ]:
# averaging filter
kernel_5 = np.ones((5, 5), np.float32) / 25
image_k5 = cv2.filter2D(image_gray, -1, kernel_5)
# blured_image = cv2.blur(img,(5,5))
image_b5 = cv2.blur(image_gray, (5, 5))

In [ ]:
mse_kb = mean_squared_error(image_k5, image_b5)
(ssim_kb, diff) = structural_similarity(image_k5, image_b5, full=True)
print(mse_kb, ssim_kb)

In [ ]:
# Laplasian
kernel_lapl = np.array([[0, -10, 0],
                        [-10, 40, -10],
                        [0, -10, 0]], np.float32)

In [ ]:
image_lapl = cv2.filter2D(image_gray, -1, kernel_lapl)

In [ ]:
plt.imshow(image_lapl, cmap="gray")

In [ ]:
# ДЗ 2
# Зашумить изображение при помощи шума гаусса, постоянного шума.
# Протестировать медианный фильтр, фильтр гаусса, билатериальный фильтр, фильтр нелокальных средних с различными параметрами.
# Выяснить, какой фильтр показал лучший результат фильтрации шума.

# Lab Work

In [ ]:
image = cv2.imread("img.jpg")
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
plt.imshow(image_gray)

In [ ]:
#Шум Гаусса
def add_gauss_noise(image, mean=0, stddev=50):
    noise = np.zeros(image.shape, np.uint8)
    cv2.randn(noise, mean, stddev)

    result = cv2.add(image, noise)
    return result


image_with_gauss = add_gauss_noise(image_gray)
plt.imshow(image)

In [ ]:
#Постоянный шум
add_constant_noise = lambda image, value: cv2.add(image, np.full(image_gray.shape, value, dtype=np.uint8))
image_with_constant = add_constant_noise(image_gray, 10)
plt.imshow(image_with_constant)

In [ ]:
filters_gauss = {}
params_list = []

# Медианный
for k in [3, 5, 7]:
    name = f'Median k={k}'
    filters_gauss[name] = cv2.medianBlur(image_with_gauss, k)
    params_list.append(name)

# Гауссовский
for k in [3, 5, 7]:
    name = f'Gaussian k={k}'
    filters_gauss[name] = cv2.GaussianBlur(image_with_gauss, (k, k), 0)
    params_list.append(name)

# Билатеральный
for d, sigma in [(5, 50), (7, 75), (9, 100)]:
    name = f'Bilateral d={d}, σ={sigma}'
    filters_gauss[name] = cv2.bilateralFilter(image_with_gauss, d, sigma, sigma)
    params_list.append(name)

# фильтр нелокальных средних с различными параметрами
for h in [10, 20, 30]:
    name = f'NLM h={h}'
    filters_gauss[name] = cv2.fastNlMeansDenoising(image_with_gauss, h=h)
    params_list.append(name)

print("ГАУССОВСКИЙ ШУМ")

results_gauss = []
for name, filtered in filters_gauss.items():
    mse = mean_squared_error(image_gray, filtered)
    ssim_val, _ = structural_similarity(image_gray, filtered, full=True)
    results_gauss.append((name, mse, ssim_val))
    print(f"{name:30s} | MSE: {mse:8.2f} | SSIM: {ssim_val:.4f}")
best_gauss = max(results_gauss, key=lambda x: x[2])
print(f"\nЛучший: {best_gauss[0]} | MSE: {best_gauss[1]:.2f} | SSIM: {best_gauss[2]:.4f}")

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(16, 16))
axes = axes.flatten()

axes[0].imshow(image_gray, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(image_with_gauss, cmap='gray')
axes[1].set_title('With Gaussian Noise')
axes[1].axis('off')

for idx, (name, img) in enumerate(filters_gauss.items(), start=2):
    axes[idx].imshow(img, cmap='gray')
    axes[idx].set_title(name)
    axes[idx].axis('off')

for idx in range(len(filters_gauss) + 2, len(axes)):
    axes[idx].axis('off')

plt.suptitle('Фильтрация гауссовского шума', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
filters_const = {}

for k in [3, 5, 7]:
    name = f'Median k={k}'
    filters_const[name] = cv2.medianBlur(image_with_constant, k)

for k in [3, 5, 7]:
    name = f'Gaussian k={k}'
    filters_const[name] = cv2.GaussianBlur(image_with_constant, (k, k), 0)

for d, sigma in [(5, 50), (7, 75), (9, 100)]:
    name = f'Bilateral d={d}, σ={sigma}'
    filters_const[name] = cv2.bilateralFilter(image_with_constant, d, sigma, sigma)

for h in [10, 20, 30]:
    name = f'NLM h={h}'
    filters_const[name] = cv2.fastNlMeansDenoising(image_with_constant, h=h)

print("ПОСТОЯННЫЙ ШУМ")

results_const = []
for name, filtered in filters_const.items():
    mse = mean_squared_error(image_gray, filtered)
    ssim_val, _ = structural_similarity(image_gray, filtered, full=True)
    results_const.append((name, mse, ssim_val))
    print(f"{name:30s} | MSE: {mse:8.2f} | SSIM: {ssim_val:.4f}")

best_const = max(results_const, key=lambda x: x[2])
print(f"\nЛучший: {best_const[0]} | MSE: {best_const[1]:.2f} | SSIM: {best_const[2]:.4f}")

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(16, 16))
axes = axes.flatten()

axes[0].imshow(image_gray, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(image_with_constant, cmap='gray')
axes[1].set_title('With Constant Noise')
axes[1].axis('off')

for idx, (name, img) in enumerate(filters_const.items(), start=2):
    axes[idx].imshow(img, cmap='gray')
    axes[idx].set_title(name)
    axes[idx].axis('off')

for idx in range(len(filters_const) + 2, len(axes)):
    axes[idx].axis('off')

plt.suptitle('Фильтрация постоянного шума', fontsize=16)
plt.tight_layout()
plt.show()